# Download annual per capita nightlights 

Create a csv file with annual per capita nightlight values over DHS clusters, for years 2000 to 2013.

Import, authenticate and initialize the earth-engine library

In [1]:
import ee

ee.Authenticate()

ee.Initialize()

ModuleNotFoundError: No module named 'ee'

In [2]:
# Import other libraries
import csv

Read the csv file with survey points

In [3]:
data_dir = '/mimer/NOBACKUP/groups/globalpoverty1/cindy/eoml_ch_wb/data/interim'
dhs_cluster_file_path = os.path.join(data_dir, 'dhs_treat_control_confounders.csv')
df = pd.read_csv(dhs_cluster_file_path)
df.head()

,country,survey_start_year,year,lat,lon,GID_1,GID_2,households,rural,iwi
0,angola,2006,2006,-12.169283,13.859255,AGO.2,AGO.2.9,20,0,68.096300
1,angola,2006,2006,-12.361700,14.778201,AGO.2,AGO.2.2,19,1,10.609053
2,angola,2006,2006,-10.827451,14.342114,AGO.7,AGO.7.1,25,1,29.973928
3,angola,2006,2006,-9.991311,14.898295,AGO.7,AGO.7.5,1,1,7.704135
4,angola,2006,2006,-11.518893,14.750437,AGO.7,AGO.7.10,24,1,13.371826


Function to create a feature with a 6.7km square around a lat/lon

In [ ]:
function createDHSFeature(row) {
  lat = ee.Number(row.get('lat'));
  lon = ee.Number(row.get('lon'));
  dhs_id = row.get('dhs_id');
  properties = {'dhs_id':dhs_id,
               'lat':lat,
               'lon':lon}
    
  #Calc coordinates of a 6.7km bounding box around the point
  minx = lon - 3350
  maxx = lon + 3350
  miny = lat - 3350
  maxy = lat + 3350
  roi = ee.Geometry.Rectangle([minx,miny,maxx,maxy])
  
  return ee.Feature(geometry, properties);
}

Load nightlight and population density collections

In [1]:
africa_region = ee.Geometry.Polygon([
    [-20, 38],   
    [55, 38],    
    [55, -36],   
    [-20, -36]   
])

dmsp = ee.ImageCollection('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS').filterBounds(africa_region)
pop_count = ee.ImageCollection(ee.ImageCollection("CIESIN/GPWv411/GPW_Population_Count")).filterBounds(africa_region)

#get a composite image for each nighlight year
dmsp2000img = dmsp.filterDate('2000-01-01', '2000-12-31').select([stable_lights]).median()
dmsp2001img = dmsp.filterDate('2001-01-01', '2001-12-31').select([stable_lights]).median()
dmsp2002img = dmsp.filterDate('2002-01-01', '2002-12-31').select([stable_lights]).median()
dmsp2003img = dmsp.filterDate('2003-01-01', '2003-12-31').select([stable_lights]).median()
dmsp2004img = dmsp.filterDate('2004-01-01', '2004-12-31').select([stable_lights]).median()
dmsp2005img = dmsp.filterDate('2005-01-01', '2005-12-31').select([stable_lights]).median()
dmsp2006img = dmsp.filterDate('2006-01-01', '2006-12-31').select([stable_lights]).median()
dmsp2007img = dmsp.filterDate('2007-01-01', '2007-12-31').select([stable_lights]).median()
dmsp2008img = dmsp.filterDate('2008-01-01', '2008-12-31').select([stable_lights]).median()
dmsp2009img = dmsp.filterDate('2009-01-01', '2009-12-31').select([stable_lights]).median()
dmsp2010img = dmsp.filterDate('2010-01-01', '2010-12-31').select([stable_lights]).median()
dmsp2011img = dmsp.filterDate('2011-01-01', '2011-12-31').select([stable_lights]).median()
dmsp2012img = dmsp.filterDate('2012-01-01', '2012-12-31').select([stable_lights]).median()
dmsp2013img = dmsp.filterDate('2013-01-01', '2013-12-31').select([stable_lights]).median()

#pop counts are already just one image: get three years available
pop_count2000img = pop_count.filterDate('2000-01-01', '2000-12-31')
pop_count2005img = pop_count.filterDate('2005-01-01', '2005-12-31')
pop_count2010img = pop_count.filterDate('2010-01-01', '2010-12-31')

NameError: name 'ee' is not defined

In [ ]:
Function to calculate sum of viirs2000 within a region

In [ ]:
def calculate_dmsp_sum(feature):
    nl2000 = dmsp2000img.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=feature.geometry(),
        scale=927.67
    )
    return feature.set(nl2000)

Open csv file, write header, loop over years and DHS points

In [5]:
# Loop through each DHS cluster and add it to a feature collection
feature_list = []
for dhs in df:
    feature = create_point_feature(row)
    feature_list.append(feature)

dhs_f_collection = ee.FeatureCollection(feature_list) 
print('Before:',dhs_f_collection.getInfo())   
dhs_f_collection2 = dhs_f_collection.map(calculate_dmsp_sum)
print('After:',dhs_f_collection2.getInfo())    

#Open csv file to write results
# csv_filename = os.path.join(data_dir,"dhs_per_capita_nightlights.csv")
# with open(csv_filename, mode="w", newline="") as csv_file:
#     csv_writer = csv.writer(csv_file)
#     csv_writer.writerow(["DHS ID","Year", "Latitude", "Longitude", "PopCount2000","PopCount2005","PopCount2010,
#                          "SumNL2000","SumNL2001","SumNL2002","SumNL2003","SumNL2004","SumNL2005",
#                          "SumNL2006","SumNL2007","SumNL2008","SumNL2009","SumNL2010","SumNL2011",
#                          "SumNL2012","SumNL2013"])
    
   
#     var featuresWithMeans = image.reduceRegions({
#   collection: maineCounties,                       
#   reducer: ee.Reducer.mean(),
#   scale: 30,
#   tileScale: 4
# });
       
#         pc2000 = pop_count2000.reduceRegion(
#             reducer=ee.Reducer.sum(),
#             geometry=roi,
#             scale=927.67,  # Resolution of the nighttime lights data in meters per pixel
#             maxPixels=1e9  # Adjust as needed
#         )

#         for download_year in range(2000, 2014):
#     print("Starting year", download_year)
#     # Filter the nightime lights for year

#     pop_year = viirs.filterDate('2015-01-01', '2015-12-31');

#         # Get the total nighttime lights and calculate per capita nighttime lights
#         total_nightlights = stats.getNumber("your_band_name")
#         total_area = roi.area()
#         per_capita_nightlights = total_nightlights.divide(total_area)

#         # Print and write the result to the CSV file
#         print("Year:", year, "Latitude:", lat, "Longitude:", lon)
#         print("Per Capita Nighttime Lights:", per_capita_nightlights.getInfo())
#         csv_writer.writerow([year, lat, lon, per_capita_nightlights.getInfo()])

# print("Results saved to", csv_filename)